In [ ]:
pip install rasterio geopandas numpy shapely

In [ ]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np

def get_area_and_percentage_in_tif(tif_path, shapefile_path, value=1):
    # Load the shapefile
    shapefile = gpd.read_file(shapefile_path)

    # Load the raster file
    with rasterio.open(tif_path) as src:
        # Ensure CRS matches, and reproject shapefile if needed
        shapefile = shapefile.to_crs(src.crs)

        # Mask the raster with the shapefile's geometry
        geometries = shapefile.geometry
        out_image, out_transform = mask(src, geometries, crop=True)

        # Extract the data band (assuming single band raster)
        data = out_image[0]

        # Calculate the pixel area in square meters (assuming CRS is in meters)
        pixel_area_sqm = src.res[0] * src.res[1]  # pixel width * pixel height in meters

        # Calculate total area within the masked region
        total_pixels = np.count_nonzero(data != src.nodata)
        total_area_sqm = total_pixels * pixel_area_sqm

        # Count the number of pixels with the given value (e.g., 1)
        pixel_count_value = np.sum(data == value)

        # Calculate the area for the specified value in square meters
        area_value_sqm = pixel_count_value * pixel_area_sqm

        # Convert both areas to square kilometers
        total_area_sqkm = total_area_sqm / 1e6
        area_value_sqkm = area_value_sqm / 1e6

        # Calculate the percentage
        percentage = (area_value_sqkm / total_area_sqkm) * 100 if total_area_sqkm > 0 else 0

        return area_value_sqkm, percentage



In [ ]:
# Example usage
tif_path = 'path/to/your/raster.tif'
shapefile_path = 'path/to/your/shapefile.shp'

area_sqkm, percentage = get_area_and_percentage_in_tif(tif_path, shapefile_path, value=1)
print(f"Total Cropland: {area_sqkm:.4f} square kilometers")
print(f"Percentage of Cropland: {percentage:.2f}%")